In [33]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [34]:
dataset=pd.read_csv("Social_Network_Ads.csv")

In [35]:
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [36]:
dataset=pd.get_dummies(dataset,drop_first=True)

In [37]:
dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,1
1,15810944,35,20000,0,1
2,15668575,26,43000,0,0
3,15603246,27,57000,0,0
4,15804002,19,76000,0,1
...,...,...,...,...,...
395,15691863,46,41000,1,0
396,15706071,51,23000,1,1
397,15654296,50,20000,1,0
398,15755018,36,33000,0,1


In [38]:
dataset["Purchased"].value_counts()

0    257
1    143
Name: Purchased, dtype: int64

In [39]:
dataset.columns

Index(['User ID', 'Age', 'EstimatedSalary', 'Purchased', 'Gender_Male'], dtype='object')

In [40]:
independent=dataset[['Age', 'EstimatedSalary','Gender_Male']]

In [41]:
independent

,Age,EstimatedSalary,Gender_Male
0,19,19000,1
1,35,20000,1
2,26,43000,0
3,27,57000,0
4,19,76000,1
...,...,...,...
395,46,41000,0
396,51,23000,1
397,50,20000,0
398,36,33000,1


In [42]:
dependent=dataset[['Purchased']]

In [43]:
dependent

,Purchased
0,0
1,0
2,0
3,0
4,0
...,...
395,1
396,1
397,1
398,0


In [44]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(independent,dependent,test_size=1/3,random_state=0)

In [45]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
X_train=sc.fit_transform(X_train)
X_test=sc.transform(X_test)

In [46]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
param_grid={'solver':['lbfgs','liblinear','saga'],'penalty':['l2']
        
            }
grid=GridSearchCV(LogisticRegression(),param_grid,refit=True,verbose=3,n_jobs=-1,scoring='f1_weighted')
grid.fit(X_train,y_train)

Fitting 5 folds for each of 3 candidates, totalling 15 fits


C:\Users\yuvar\Anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GridSearchCV(estimator=LogisticRegression(), n_jobs=-1,
             param_grid={'penalty': ['l2'],
                         'solver': ['lbfgs', 'liblinear', 'saga']},
             scoring='f1_weighted', verbose=3)

In [47]:
result=grid.cv_results_


In [48]:
grid_pred=grid.predict(X_test)

In [49]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test,grid_pred)


In [50]:
from sklearn.metrics import classification_report
clf_report=classification_report(y_test,grid_pred)
print(clf_report)

              precision    recall  f1-score   support

           0       0.89      0.93      0.91        85
           1       0.87      0.80      0.83        49

    accuracy                           0.88       134
   macro avg       0.88      0.86      0.87       134
weighted avg       0.88      0.88      0.88       134



In [51]:
from sklearn.metrics import f1_score
f_score=f1_score(y_test,grid_pred,average='weighted')
print("the f1 value for best parameter{}".format(grid.best_params_),f_score)

the f1 value for best parameter{'penalty': 'l2', 'solver': 'liblinear'} 0.8794289739855382


In [52]:
from sklearn.metrics import roc_auc_score #receiver operating characterstic _ area under curve score
roc_auc_score(y_test,grid.predict_proba(X_test)[:,1])

0.9507803121248499

In [53]:
table=pd.DataFrame.from_dict(result)
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_penalty,param_solver,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.015597,0.008950,0.001804,0.002545,l2,lbfgs,"{'penalty': 'l2', 'solver': 'lbfgs'}",0.799620,0.811321,0.68205,0.92351,0.922185,0.827737,0.089864,2
1,0.006302,0.006617,0.009298,0.007669,l2,liblinear,"{'penalty': 'l2', 'solver': 'liblinear'}",0.804764,0.811321,0.68205,0.92351,0.922185,0.828766,0.089565,1
2,0.009102,0.009720,0.002403,0.004806,l2,saga,"{'penalty': 'l2', 'solver': 'saga'}",0.799620,0.811321,0.68205,0.92351,0.922185,0.827737,0.089864,2


In [54]:
print("The clf_report value for best parameter {}".format(grid.best_params_),clf_report)

The clf_report value for best parameter {'penalty': 'l2', 'solver': 'liblinear'}               precision    recall  f1-score   support

           0       0.89      0.93      0.91        85
           1       0.87      0.80      0.83        49

    accuracy                           0.88       134
   macro avg       0.88      0.86      0.87       134
weighted avg       0.88      0.88      0.88       134

